In [18]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

features=['draw','addedWeight','declaredHorseWeight','horse_winPer',
                 'horse_winCount','jockey_winPer','jockey_winCount','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace']
target=['finishTime']
file0 = "/Users/apple/Workspace/intelliJ/jockey/history.csv"

#file1 = "/Users/apple/Workspace/intelliJ/jockey/predict.csv"
#tableName="newrace"
file1 = "/Users/apple/Workspace/intelliJ/jockey/regress.csv"
tableName="racecard"

dataset0 = pd.read_csv(file0)
dataset1 = pd.read_csv(file1)

In [20]:
allDates = dataset1.rateDate.unique()
allDates.sort()
for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset1[(dataset1['rateDate']==raceDate)]
    allSeqs = dataset_ofADay.raceSeqOfDate.unique()
    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset1['raceSeqOfDate']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['rateDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
       


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #      % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))

        #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=X_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['rateDate','raceSeqOfDate','distance','raceClass','horseId']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #dataset_test_withPredicted

        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['rateDate'])
            raceSeqOfDate =str(row['raceSeqOfDate'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDate'] ,",", row['horseId'] )

20190130
1 1200 5
1
1  record(s) updated 1 , V414
2
1  record(s) updated 1 , V302
3


/Users/apple/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


1  record(s) updated 1 , T256
4
1  record(s) updated 1 , A059
5
1  record(s) updated 1 , A255
6
1  record(s) updated 1 , S254
7
1  record(s) updated 1 , A151
8
1  record(s) updated 1 , V074
9
1  record(s) updated 1 , A348
10
1  record(s) updated 1 , A281
11
1  record(s) updated 1 , B036
12
1  record(s) updated 1 , B382
2 1200 4
1
1  record(s) updated 2 , A220
2
1  record(s) updated 2 , A266
3
1  record(s) updated 2 , T292
4
1  record(s) updated 2 , P429
5
1  record(s) updated 2 , A142
6


/Users/apple/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


1  record(s) updated 2 , S442
7
1  record(s) updated 2 , V255
8
1  record(s) updated 2 , B428
9
1  record(s) updated 2 , B155
10
1  record(s) updated 2 , C134
11
1  record(s) updated 2 , V401
12
1  record(s) updated 2 , T392
3 1650 4
1


...

1  record(s) updated 7 , B057
9
1  record(s) updated 7 , C002
10
1  record(s) updated 7 , B309
11
1  record(s) updated 7 , A187
12
1  record(s) updated 7 , C212
9 1800 3
1
0  record(s) updated 9 , B094
2
0  record(s) updated 9 , C084
3
0  record(s) updated 9 , B189
4
0  record(s) updated 9 , C080
5


/Users/apple/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


0  record(s) updated 9 , B350
6
0  record(s) updated 9 , T098
7
0  record(s) updated 9 , C049
8
0  record(s) updated 9 , A172
9
0  record(s) updated 9 , B336
10
0  record(s) updated 9 , B268
11
0  record(s) updated 9 , T281
10 1600 2
1
0  record(s) updated 10 , B387
2


/Users/apple/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


0  record(s) updated 10 , B105
3
0  record(s) updated 10 , T375
4
0  record(s) updated 10 , B457
5
0  record(s) updated 10 , C114
6
0  record(s) updated 10 , T262
7
0  record(s) updated 10 , C146
8
0  record(s) updated 10 , A367
9
0  record(s) updated 10 , V118
10
0  record(s) updated 10 , C071
11
0  record(s) updated 10 , B351
12
0  record(s) updated 10 , C197
13
0  record(s) updated 10 , A098
